In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train_DS = pd.read_csv('/kaggle/input/titanic/train.csv')
test_DS = pd.read_csv('/kaggle/input/titanic/test.csv')
train_DS.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_DS['Fam_size'] = train_DS['SibSp'] + train_DS['Parch'] + 1
test_DS['Fam_size'] = test_DS['SibSp'] + test_DS['Parch'] + 1

train_DS['Fam_type'] = pd.cut(train_DS.Fam_size, [0,1,4,7,11], labels=['Solo', 'Small', 'Big', 'Very big'])
test_DS['Fam_type'] = pd.cut(test_DS.Fam_size, [0,1,4,7,11], labels=['Solo', 'Small', 'Big', 'Very big'])

In [5]:
Y = train_DS['Survived']
cols = ['Pclass', 'Fare', 'Sex', 'Embarked', 'Fam_type']
X = train_DS[cols]
X_test = test_DS[cols]

In [6]:
one_hot_training = pd.get_dummies(X)
one_hot_test = pd.get_dummies(X_test)
final_train, final_test = one_hot_training.align(one_hot_test, join='left', axis=1)

In [7]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(final_train, Y)

In [8]:
from sklearn.impute import SimpleImputer
simp = SimpleImputer()
final_train = simp.fit_transform(Xtrain)

In [9]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(final_train, Ytrain)

DecisionTreeClassifier()

In [10]:
final_test.fillna(0)
final_test=final_test.replace(np.nan, 0)

In [11]:
pred = model.predict(final_test)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [12]:
pred.shape

(418,)

In [13]:
output = pd.DataFrame({'PassengerId': test_DS.PassengerId, 'Survived': pred})
output

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
